# Creating basic inputs and managing the outputs

#### Import required packages

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
from langchain.llms import OpenAI

In [3]:
api_key = os.environ['OPENAI_API_KEY']

In [3]:
llm = OpenAI(openai_api_key = api_key)

In [4]:
# text completion
print(llm("Here's the fact about Galileo: "))



Galileo Galilei was an Italian astronomer, physicist, engineer, and philosopher who played a major role in the Scientific Revolution of the 17th century. He is best known for his improvements to the telescope and consequent astronomical observations, and for providing evidence that supported the Copernican theory that Earth and other planets revolve around the Sun. He was tried by the Inquisition and found guilty of heresy, and was placed under house arrest for the remainder of his life.


In [5]:
# Multiple completions
result = llm.generate(["Here's a fact about Steve Jobs: ", 
                      "Here's a fact about John Lennon: "])

In [6]:
# Details of the output
result.schema()

{'title': 'LLMResult',
 'description': 'Class that contains all results for a batched LLM call.',
 'type': 'object',
 'properties': {'generations': {'title': 'Generations',
   'type': 'array',
   'items': {'type': 'array', 'items': {'$ref': '#/definitions/Generation'}}},
  'llm_output': {'title': 'Llm Output', 'type': 'object'},
  'run': {'title': 'Run',
   'type': 'array',
   'items': {'$ref': '#/definitions/RunInfo'}}},
 'required': ['generations'],
 'definitions': {'Generation': {'title': 'Generation',
   'description': 'A single text generation output.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'generation_info': {'title': 'Generation Info', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'Generation',
     'enum': ['Generation'],
     'type': 'string'}},
   'required': ['text']},
  'RunInfo': {'title': 'RunInfo',
   'description': 'Class that contains metadata for a single execution of a Chain or model.',
   'type': '

In [7]:
result.llm_output

{'token_usage': {'completion_tokens': 93,
  'total_tokens': 111,
  'prompt_tokens': 18},
 'model_name': 'text-davinci-003'}

In [8]:
result.generations

[[Generation(text="\n\nSteve Jobs was the co-founder and CEO of Apple Inc., one of the world's most successful and influential technology companies. He was also the primary driving force behind the development of revolutionary products such as the iPhone, iPad, and iPod.", generation_info={'finish_reason': 'stop', 'logprobs': None})],
 [Generation(text='\n\nJohn Lennon was a founding member of the Beatles, an iconic English rock band, and wrote some of their most iconic songs, including "Imagine," "Help!," and "Strawberry Fields Forever."', generation_info={'finish_reason': 'stop', 'logprobs': None})]]

#### Chat Models

In [9]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(openai_api_key = api_key)

In [10]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [11]:
result = chat([HumanMessage(content='Can you tell me a fact about The Beatles?')])
result

AIMessage(content='One interesting fact about The Beatles is that their iconic album "Abbey Road" features a cover photo of the band walking on a zebra crossing. This photo has become one of the most famous album covers of all time and the actual zebra crossing in London has become a popular tourist attraction for Beatles fans.')

In [12]:
result.content

'One interesting fact about The Beatles is that their iconic album "Abbey Road" features a cover photo of the band walking on a zebra crossing. This photo has become one of the most famous album covers of all time and the actual zebra crossing in London has become a popular tourist attraction for Beatles fans.'

In [13]:
# Giving a personality to the LLM
result = chat.generate(
    [
        [SystemMessage(content="You're a high-school kid"),
         HumanMessage(content='Tell me a fact about Nepal')]
    ]
)

In [14]:
result

LLMResult(generations=[[ChatGeneration(text="One interesting fact about Nepal is that it is home to the world's highest mountain, Mount Everest, which stands at an impressive height of 8,848 meters (29,029 feet) above sea level. Many climbers from around the globe visit Nepal to undertake the challenging journey to the Everest Base Camp and attempt to summit this iconic peak.", generation_info={'finish_reason': 'stop'}, message=AIMessage(content="One interesting fact about Nepal is that it is home to the world's highest mountain, Mount Everest, which stands at an impressive height of 8,848 meters (29,029 feet) above sea level. Many climbers from around the globe visit Nepal to undertake the challenging journey to the Everest Base Camp and attempt to summit this iconic peak."))]], llm_output={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 23, 'total_tokens': 91}, 'model_name': 'gpt-3.5-turbo'}, run=[RunInfo(run_id=UUID('2e7ee966-13d0-43ae-8372-3f76c3ba1ccc'))])

In [15]:
result.llm_output

{'token_usage': {'completion_tokens': 68,
  'prompt_tokens': 23,
  'total_tokens': 91},
 'model_name': 'gpt-3.5-turbo'}

In [16]:
result.generations[0][0]

ChatGeneration(text="One interesting fact about Nepal is that it is home to the world's highest mountain, Mount Everest, which stands at an impressive height of 8,848 meters (29,029 feet) above sea level. Many climbers from around the globe visit Nepal to undertake the challenging journey to the Everest Base Camp and attempt to summit this iconic peak.", generation_info={'finish_reason': 'stop'}, message=AIMessage(content="One interesting fact about Nepal is that it is home to the world's highest mountain, Mount Everest, which stands at an impressive height of 8,848 meters (29,029 feet) above sea level. Many climbers from around the globe visit Nepal to undertake the challenging journey to the Everest Base Camp and attempt to summit this iconic peak."))

In [17]:
result = chat([
    HumanMessage(content='Tell me a sentence about Buddha')],
    temperature=2, presence_penalty=1, max_tokens=50 )
# Presence_penalty is used to encourage the model to include a diverse range of tokens in the generated text.
# temperature is randomness or creativity

In [18]:
result.content

"Buddha's teachings found in Buddhism are centered around enlightenment and inner-self development."

#### Caching can be used to store historical responses to generate faster later

In [20]:
import langchain
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key = api_key)
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()
llm.predict("Tell me a fact about zero")

'One interesting fact about zero is that it is the only number that is neither positive nor negative. It is considered a neutral number, often referred to as "the absence of quantity." Zero also plays a crucial role in mathematics as the additive identity, meaning any number plus zero equals the same number.'

In [3]:
# # use llamas2 for the above purpose
# import langchain
# from langchain.chat_models import ChatOllama
# llm = ChatOllama()
# from langchain.cache import InMemoryCache
# langchain.llm_cache = InMemoryCache()
# llm.predict("Tell me a fact about Sun")

# Prompt Templates

In [4]:
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key=api_key)
print(llm('Here is a fun fact about Messi:'))



Lionel Messi is the only player in history to win six Ballon d'Or awards.


In [5]:
from langchain import PromptTemplate

# An example prompt with no input variables
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a fact")
no_input_prompt.format()

'Tell me a fact'

In [6]:
# An example prompt with one input variable
one_input_prompt = PromptTemplate(input_variables=["topic"], template="Tell me a fact about {topic}.")
# Notice how the stirng "topic" gets automatically converted to a parameter name, very convienent! 
one_input_prompt.format(topic="Socrates")

'Tell me a fact about Socrates.'

In [7]:
# An example prompt with multiple input variables
multiple_input_prompt = PromptTemplate(
    input_variables=["topic", "level"], 
    template="Tell me a fact about {topic} for a student {level} level."
)
multiple_input_prompt.format(topic='Led Zepplin',level='8th Grade')

'Tell me a fact about Led Zepplin for a student 8th Grade level.'

In [8]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [9]:
system_template="You are an AI recipe assistant that specializes in {dietary_preference} dishes that can be prepared in {cooking_time}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [10]:
system_message_prompt.input_variables

['cooking_time', 'dietary_preference']

In [11]:
human_template="{recipe_request}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)